# A Transportation Problem
## Importing GAMSPy

In [1]:
from gamspy import Container, Sum, Sense

## Importing Further Python Libraries


In [2]:
import pandas as pd
import sys

## Prepare data

In [3]:
distances = pd.DataFrame([
    ["seattle", "new-york", 2.5],
    ["seattle", "chicago", 1.7],
    ["seattle", "topeka", 1.8],
    ["san-diego", "new-york", 2.5],
    ["san-diego", "chicago", 1.8],
    ["san-diego", "topeka", 1.4],
], columns=['from', 'to', 'distance'])

distances.set_index(['from', 'to'])

distance
from      to                
seattle   new-york       2.5
          chicago        1.7
          topeka         1.8
san-diego new-york       2.5
          chicago        1.8
          topeka         1.4

In [4]:
capacities = pd.DataFrame([["seattle", 350], ["san-diego", 600]], columns=['city', 'capacity'])

capacities.set_index('city')

,capacity
city,
seattle,350
san-diego,600


In [5]:
demands = pd.DataFrame([["new-york", 325], ["chicago", 300], ["topeka", 275]], columns=['city', 'demand'])

demands.set_index('city')

,demand
city,
new-york,325
chicago,300
topeka,275


## Create a Container

In [6]:
m = Container()

## Define Sets

In [7]:
i = m.addSet(name="i", records=capacities.city)
j = m.addSet(name="j", records=demands.city)

## Parameter

In [8]:
a = m.addParameter(name="a", domain=[i], records=capacities)
b = m.addParameter(name="b", domain=[j], records=demands)
d = m.addParameter(name="d", domain=[i, j], records=distances)
c = m.addParameter(name="c", domain=[i, j])
c[i, j] = 90 * d[i, j] / 1000

## Variables

In [9]:
x = m.addVariable(name="x", domain=[i, j], type="Positive")

## Equations
### Supply

In [10]:
supply = m.addEquation(name="supply", domain=[i])

supply[i] = Sum(j, x[i, j]) <= a[i]

### Demand

In [11]:
demand = m.addEquation(name="demand", domain=[j])

demand[j] = Sum(i, x[i, j]) >= b[j]

## Objective

In [12]:
obj = Sum((i, j), c[i, j] * x[i, j])

## Model

In [13]:
transport = m.addModel(
        name="transport",
        equations=m.getEquations(),
        problem="LP",
        sense=Sense.MIN,
        objective=obj,
    )

## Solve

In [14]:
transport.solve(output=sys.stdout)

--- Job _gams_py_gjo4.gms Start 10/10/23 13:04:04 45.0.0 6d29854d WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\jbroi\anaconda3\envs\test\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\jbroi\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    LP CPLEX
    MIP CPLEX
    RMIP CPLEX
    NLP CONOPT
    MCP PATH
    MPEC NLPEC
    RMPEC CONVERT
    CNS CONOPT
    DNLP CONOPT
    RMINLP CONOPT
    MINLP SBB
    QCP CONOPT
    MIQCP SBB
    RMIQCP CONOPT
    EMP CONVERT
    Restart C:\Users\jbroi\AppData\Local\Temp\tmpmgkjze5t\default_restart.g00
    Input C:\Users\jbroi\AppData\Local\Temp\tmpmgkjze5t\_gams_py_gjo4.gms
    Output C:\Users\jbroi\AppData\Local\Temp\tmpmgkjze5t\_gams_py_gjo4.lst
    Save C:\Users\jbroi\AppData\Local\Temp\tmpmgkjze5t\default_save.g00
    ScrDir C:\Users\jbroi\AppData\Local\Temp\tmpmgkjze5t\225a\
    SysDir C:\Users\jbroi\anaconda3\envs\test\Lib\site-packages\gamspy_base\
    CurDir C:\Users\jbroi\AppData\Local\Temp\tmpmgkjze5t\
    L

## Results

In [15]:
x_s = x.records.set_index(['i', 'j'])

x_s

level  marginal  lower  upper  scale
i         j                                             
seattle   new-york   50.0     0.000    0.0    inf    1.0
          chicago   300.0     0.000    0.0    inf    1.0
          topeka      0.0     0.036    0.0    inf    1.0
san-diego new-york  275.0     0.000    0.0    inf    1.0
          chicago     0.0     0.009    0.0    inf    1.0
          topeka    275.0     0.000    0.0    inf    1.0

In [16]:
transport.objective_value

153.675

In [17]:
transport.status

<ModelStatus.OptimalGlobal: 1>